# 🎓 學術主頁生成器 - Gradio 版

**一鍵生成精美學術主頁 - 無需 Node.js，純 Python 實現**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/xhh678876/personal_page/blob/main/Gradio_Academic_Homepage_Generator.ipynb)

---

## ✨ 優勢

- 🚀 **超簡單** - 只需運行兩個單元即可
- 🆓 **完全免費** - 使用 Google Colab 資源
- 🎨 **精美界面** - Gradio 自動生成的 Web UI
- 🤖 **AI 驅動** - Gemini/GPT-4o 智能解析
- 📱 **響應式** - 生成的主頁自適應所有設備

---

## 📝 使用步驟

1. **運行所有單元** - Runtime → Run all
2. **等待安裝完成** - 約 30 秒
3. **點擊生成的鏈接** - 看到類似 `Running on public URL: https://xxx.gradio.live`
4. **在 Web 界面操作**：
   - 選擇 AI 提供商（推薦 Gemini - 免費）
   - 輸入 API Key
   - 上傳簡歷 PDF
   - 點擊「生成主頁」
   - 下載生成的 HTML 文件

## 就是這麼簡單！

## 步驟 1: 安裝依賴

這會安裝所有必要的 Python 包和系統工具

In [ ]:
# 安裝 Python 包
!pip install -q gradio google-generativeai openai pdf2image pillow

# 安裝 poppler（PDF 處理工具）
!apt-get update -qq
!apt-get install -y -qq poppler-utils

print("✅ 依賴安裝完成！")
print("📦 已安裝: Gradio, Gemini AI, OpenAI, pdf2image, poppler-utils")

## 步驟 2: 啟動應用

運行此單元後，點擊生成的公網鏈接（`https://xxx.gradio.live`）

In [ ]:
import gradio as gr
import google.generativeai as genai
from openai import OpenAI
import base64
import json
from pdf2image import convert_from_path
import io
from PIL import Image
import os

# ============ PDF 處理 ============

def pdf_to_images(pdf_file):
    """將 PDF 轉換為圖片列表"""
    try:
        # Gradio 傳入的是文件路徑字符串
        print(f"正在處理 PDF: {pdf_file}")
        
        # 使用 poppler 轉換 PDF 為圖片
        images = convert_from_path(pdf_file, dpi=150)
        
        print(f"成功轉換了 {len(images)} 頁")
        return images
    except Exception as e:
        print(f"PDF 轉換錯誤: {str(e)}")
        raise Exception(f"PDF 轉換失敗: {str(e)}。請確保上傳的是有效的 PDF 文件。")

# ============ AI 解析 ============

def parse_with_gemini(images, api_key):
    """使用 Gemini Vision API 解析簡歷"""
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel('gemini-2.0-flash-exp')
    
    prompt = """分析這份學術簡歷，提取所有信息並返回 JSON 格式：
{
  "name": "姓名",
  "title": "職稱",
  "email": "郵箱",
  "website": "網站",
  "bio": "個人介紹",
  "sections": [
    {
      "title": "章節標題",
      "type": "timeline/grid-list/text-content",
      "items": [{"title": "", "subtitle": "", "date": "", "description": "", "tags": []}]
    }
  ]
}

類型選擇：timeline=時間順序，grid-list=列表展示，text-content=段落文本。返回純 JSON。"""

    response = model.generate_content([prompt] + images)
    text = response.text.strip()
    if text.startswith("```json"):
        text = text[7:-3].strip()
    elif text.startswith("```"):
        text = text[3:-3].strip()
    return json.loads(text)

def parse_with_openai(images, api_key):
    """使用 OpenAI GPT-4o Vision API 解析簡歷"""
    client = OpenAI(api_key=api_key)
    
    image_contents = []
    for img in images:
        buffered = io.BytesIO()
        img.save(buffered, format="PNG")
        img_str = base64.b64encode(buffered.getvalue()).decode()
        image_contents.append({"type": "image_url", "image_url": {"url": f"data:image/png;base64,{img_str}"}})
    
    prompt = """分析這份學術簡歷，提取所有信息並返回 JSON 格式。"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": [{"type": "text", "text": prompt}, *image_contents]}],
        max_tokens=4000
    )
    
    text = response.choices[0].message.content.strip()
    if text.startswith("```json"):
        text = text[7:-3].strip()
    return json.loads(text)

# ============ HTML 生成 ============

def generate_html(data):
    """生成精美的 HTML 主頁（簡化版）"""
    sections_html = ""
    
    for section in data.get("sections", []):
        section_type = section.get("type", "text-content")
        title = section.get("title", "")
        items = section.get("items", [])
        
        if section_type == "timeline":
            items_html = "".join([
                f'''<div class="item"><h3>{item.get('title', '')}</h3><p>{item.get('subtitle', '')}</p><p class="date">{item.get('date', '')}</p></div>'''
                for item in items
            ])
        elif section_type == "grid-list":
            items_html = "".join([
                f'''<div class="card"><h3>{item.get('title', '')}</h3><p>{item.get('description', '')}</p></div>'''
                for item in items
            ])
        else:
            items_html = "".join([f"<p>{item.get('description', '')}</p>" for item in items])
        
        sections_html += f'<section><h2>{title}</h2>{items_html}</section>'
    
    return f"""<!DOCTYPE html>
<html><head><meta charset="UTF-8"><title>{data.get('name', '')}</title>
<style>
* {{margin:0;padding:0;box-sizing:border-box}}
body{{font-family:-apple-system,sans-serif;background:linear-gradient(135deg,#667eea,#764ba2);padding:2rem}}
.container{{max-width:1200px;margin:0 auto;background:rgba(255,255,255,0.95);border-radius:20px;padding:3rem}}
.hero{{text-align:center;padding:2rem 0;border-bottom:2px solid #eee;margin-bottom:2rem}}
h1{{font-size:3rem;background:linear-gradient(135deg,#667eea,#764ba2);-webkit-background-clip:text;-webkit-text-fill-color:transparent}}
h2{{color:#667eea;margin:2rem 0 1rem}}
.item,.card{{background:#f8f9fa;padding:1rem;margin:1rem 0;border-radius:8px}}
</style></head>
<body><div class="container">
<div class="hero"><h1>{data.get('name', '')}</h1><p>{data.get('title', '')}</p><p>{data.get('bio', '')}</p></div>
{sections_html}
</div></body></html>"""

# ============ Gradio 處理函數 ============

def process_resume(pdf_file, provider, api_key, progress=gr.Progress()):
    if pdf_file is None:
        return None, "❌ 請上傳 PDF 文件", None
    if not api_key:
        return None, "❌ 請輸入 API Key", None
    
    try:
        progress(0.2, desc="📄 轉換 PDF...")
        images = pdf_to_images(pdf_file)
        
        progress(0.4, desc=f"🤖 使用 {provider} 解析...")
        data = parse_with_gemini(images, api_key) if provider == "Gemini" else parse_with_openai(images, api_key)
        
        progress(0.8, desc="✨ 生成主頁...")
        html = generate_html(data)
        
        output_path = "academic_homepage.html"
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(html)
        
        return output_path, "✅ 生成成功！", json.dumps(data, ensure_ascii=False, indent=2)
    except Exception as e:
        return None, f"❌ 錯誤: {str(e)}", None

# ============ 創建界面 ============

with gr.Blocks(title="🎓 學術主頁生成器", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 🎓 學術主頁生成器
    > AI 驅動的學術主頁自動生成 - Gemini / GPT-4o Vision
    
    ### 📝 步驟：選擇提供商 → 輸入 Key → 上傳 PDF → 生成 → 下載
    """)
    
    with gr.Row():
        with gr.Column(scale=1):
            provider = gr.Radio(["Gemini", "OpenAI"], value="Gemini", label="🤖 AI 提供商", info="推薦 Gemini（免費）")
            api_key = gr.Textbox(label="🔑 API Key", type="password", placeholder="請輸入 API Key", info="Gemini: https://aistudio.google.com/app/apikey")
            pdf_file = gr.File(label="📄 上傳簡歷 PDF", file_types=[".pdf"], type="filepath")
            submit_btn = gr.Button("✨ 生成主頁", variant="primary", size="lg")
        
        with gr.Column(scale=1):
            status = gr.Textbox(label="📊 狀態", interactive=False)
            html_file = gr.File(label="📥 下載生成的主頁")
            json_output = gr.Code(label="📋 提取的數據", language="json")
    
    submit_btn.click(process_resume, [pdf_file, provider, api_key], [html_file, status, json_output])
    
    gr.Markdown("""
    ---
    ### 📝 API Key 獲取
    - **Gemini（推薦）**: https://aistudio.google.com/app/apikey
    - **OpenAI**: https://platform.openai.com/api-keys
    
    **Made with ❤️ for Researchers** | Powered by Gemini / GPT-4o Vision
    """)

# 啟動應用！
demo.launch(share=True, debug=True)

## 🎉 完成！

運行上面的單元後，會出現兩個鏈接：
- **本地鏈接**：`http://127.0.0.1:7860`
- **公網鏈接**：`https://xxxxxxxx.gradio.live` ← **點擊這個！**

在 Gradio 界面中操作即可！

---

### 📚 API Key 獲取

- **Gemini（推薦）**：https://aistudio.google.com/app/apikey
- **OpenAI**：https://platform.openai.com/api-keys

---

### ❓ 常見問題

**Q: PDF 轉換失敗？**  
A: 確保上傳的是有效的 PDF 文件。如果還是失敗，重新運行步驟 1 確保 poppler-utils 安裝成功。

**Q: 為什麼選擇 Gradio？**  
A: Gradio 是純 Python，不需要 Node.js，在 Colab 上更簡單直接。

**Q: 會話斷開了怎麼辦？**  
A: 重新運行所有單元即可，約 1 分鐘。

---

**GitHub**: https://github.com/xhh678876/personal_page